# Merging GRASS outputs and then cep attributes to the merged output
execution time ~= 1 min

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
output_directory = os.environ['GRASS_OUTPUT_DIRECTORY']
cep_attributes_path = os.environ['CEP_ATTRIBUTES_CSV_PATH']

In [3]:
class MergingDataframes():
    """
    Contains functions Merge the outputs of the GRASS script into a single dataframe with the CEP attributes
    """
    def __init__(self, output_directory,cep_attributes_path):
        self.df = None
        self.qid_df = None
        self.output_directory = output_directory
        self.cep_attributes_path = cep_attributes_path
    

    def aggregate_grass_outputs(self):
        """
        Aggregate the outputs of the GRASS script into a single dataframe with the CEP attributes
        """
        dataframes = []
        # step through all csv files in the output directory
        for csv_file in os.listdir(self.output_directory):
            if csv_file.endswith('.csv'):
                # if csv file empty, skip
                if os.path.getsize(os.path.join(output_directory, csv_file)) == 0:
                    continue
                columns = ['transition_band','cep_id', 'area']
                df = pd.read_csv(os.path.join(output_directory, csv_file), names=columns)
                df['qid'] = csv_file.split('.')[0]
                # for each csv append to list of dataframes
                dataframes.append(df)
               
        # concatenate all dataframes into multi-indexed dataframe (cep_id, transition_band)
        df = pd.concat(dataframes)
        # group by cep_id and transition_band and sum the areas (to create a multi-indexed dataframe with sum of areas)
        df = df.groupby(['cep_id', 'qid', 'transition_band']).sum()
        #transpose the dataframe so that the transition bands are columns and fill NaN with 0 and set column names to transition_i
        df = df.unstack().fillna(0)
        #fix column names to remove multi-index
        df.columns = df.columns.droplevel()
        df.columns = [f'transition_{i}' for i in range(11)]

        # make unique quantile ids for each quantile name e.g 1 = 0E_0N
        quantiles = df.index.get_level_values('qid').unique()
        quantiles = dict(zip(quantiles, range(len(quantiles))))
        df = df.reset_index()
        df['qid'] = df['qid'].map(quantiles)

        # make separate df for quantiles 
        self.qid_df = pd.DataFrame.from_dict(quantiles, orient='index', columns=['qid']).reset_index().rename(columns={'index':'quantile_name'}).set_index('qid')

        self.df = df.set_index(['cep_id','qid'])

    def merge_cep_attributes(self, cep_df, to_csv=False):
        """
        Merge the aggregated outputs of the GRASS script with the CEP attributes
        :param cep_df: dataframe containing the CEP attributes
        :param to_csv: if True, save the final dataframe to a csv file
        """
        # join the two dataframes on cep_id and cid to get the attributes
        df = self.df.join(cep_df, on='cep_id')
        df['eco'] = pd.to_numeric(df['eco'], errors='coerce').astype('Int64')
        df['country'] = pd.to_numeric(df['country'], errors='coerce').astype('Int64')
        self.df = df

        if to_csv:
            self.df.to_csv('final_output.csv')
            self.qid_df.to_csv('qid.csv')
        

    # def find_tile_with_id(self, cep_id, output_directory):
    #     # step through all csv files in the output directory
    #     for csv_file in os.listdir(output_directory):
    #         if csv_file.endswith('.csv'):
    #             # if csv file empty, skip
    #             if os.path.getsize(os.path.join(output_directory, csv_file)) == 0:
    #                 continue
    #             columns = ['transition_band','cep_id', 'area']
    #             df = pd.read_csv(os.path.join(output_directory, csv_file), names=columns)

 # Fixing CEP df

In [4]:
class FixingCEPAttributes():
    def __init__(self, cep_attributes_path):
        self.cep_df = pd.read_csv(cep_attributes_path, index_col='cid')

    def fix_NaN_country(self):
        """
        Fix the missing country values in the CEP attributes
        """
        # get rows where NaN or null country values
        missing_country = self.cep_df[self.cep_df['country_name'].isnull()].index.values

        # There should only be one missing country which is cid 1 and we can manually update it
        if len(missing_country) == 1:
            # update cid 1 with country (code): 171, country name: Lithuania, iso3: LTU
            self.cep_df.loc[1, 'country'] = 171
            self.cep_df.loc[1, 'country_name'] = 'Lithuania'
            self.cep_df.loc[1, 'iso3'] = 'LTU'

    def fix_missing_ids(self):
        """
        In the cid_index.csv there is one missing id when compared to the raster tiff files from 1 to 463710 ids (inclusive) which is 295147
        after investing the data in QGIS we found that the missing id is a single pixel in the conifer forests of Russia (should be the same as cid 339212)
        """
        # in cep_df check if there are any missing ids from 1 to 463710 ids (inclusive)
        missing_ids = set(range(1, 463711)) - set(self.cep_df.index)
        missing_ids # {cid: 295147}

        if missing_ids == {295147}:
            # maunally insert missing record for id 295147 (copy from 339212 with different id)
            self.cep_df.loc[295147] = self.cep_df.loc[339212].copy()

    def concatenate_PAs(self):
        """
        Concatenate the PAs of the same cid, country and ecoregion into a single row
        """
        dupes = self.cep_df.reset_index().groupby(['cid','country', 'eco']).filter(lambda x: len(x) > 1)
        dupes = dupes.groupby(['cid','country', 'eco']).agg(
            {
                'country_name':'first',
                'iso3':'first',
                'eco_name':'first',
                'is_marine':'first',
                # concatenate pa ids and pa names into strings NOT lists
                'pa': lambda x: ','.join(map(str, x)),
                'pa_name': lambda x: ','.join(map(str, x)),
                'is_protected': 'first'
            })
        dupes = dupes.reset_index()

        non_dupes = self.cep_df.reset_index().groupby(['cid','country', 'eco']).filter(lambda x: len(x) == 1)

        # combine dupes and non_dupes together
        self.cep_df = pd.concat([non_dupes, dupes])
        self.cep_df = self.cep_df.set_index('cid')
        self.cep_df

    def verify_fixes(self):

        missing_countries = self.cep_df[self.cep_df['country_name'].isnull()].index.values
        if len(missing_countries) != 0:
            print(f'Missing countries: {missing_countries}')

        # get max index
        max_index = self.cep_df.index.max()
        if len(self.cep_df) != max_index: 
            print(f'Index does not match cep_ids, max index: {max_index}, number of rows: {len(self.cep_df)}')
            missing_ids = set(range(1, max_index)) - set(self.cep_df.index) # should be empty
            print(f'Missing ids: {missing_ids}')

        print('All fixes verified')

    def fix_all(self):
        self.fix_NaN_country()
        self.fix_missing_ids()
        self.concatenate_PAs()
        self.verify_fixes()
        return self.cep_df


In [5]:
# estimated time to run: 27 seconds
cep_df = FixingCEPAttributes(cep_attributes_path).fix_all()
cep_df.head()

All fixes verified


,country,country_name,iso3,eco,eco_name,is_marine,pa,pa_name,is_protected
cid,,,,,,,,,
1,171,Lithuania,LTU,80412,Central European mixed forests,False,0,NaN,False
2,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,0,NaN,False
3,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,478191,South Orkney Islands Southern Shelf Marine Pro...,True
4,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,555547601,South Georgia and South Sandwich Islands Marin...,True
5,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,555624810,Ross Sea Region Marine Protected Area,True


In [6]:
# aggregate GRASS outputs into df
merger = MergingDataframes(output_directory,cep_attributes_path)
merger.aggregate_grass_outputs()

In [7]:
#merge cep attributes with GRASS outputs
merger.merge_cep_attributes(cep_df, to_csv=False)

In [48]:
# get all the PAs from merged dataframe 
# make a list of all pas in the cep_df
merger_pa = {str(x) for x in merger.df['pa'] }

# Flatten the set and convert each number to an individual element
merger_pa = {int(item) for sublist in merger_pa for item in sublist.split(',')}

print(len(merger_pa))


275887


In [38]:
# make a list of all pas in the cep_df
cep_pa = {x for x in cep_df['pa'] }
print(len(cep_pa))

376512


In [43]:
# check for duplicates in cep_pa only
duplicates = set()
for x in cep_pa:
    if x not in duplicates:
        duplicates.add(x)

print(len(duplicates))

376512


In [51]:
# get difference between the two sets
diff = merger_pa - cep_pa
print(len(diff))
print(diff)

57725
{555745287, 555533538, 24, 555732260, 312626, 555533539, 555745326, 555745329, 555732264, 555745331, 555533540, 73, 555732275, 555533542, 312644, 555732286, 555614373, 555732287, 555614375, 555732290, 555732291, 187, 555732295, 555614416, 219, 220, 555654144, 555614448, 242, 253, 555732310, 309, 555732319, 330, 331, 555732320, 336, 340, 555732322, 393575, 555533552, 393584, 393585, 393590, 393593, 393601, 393602, 555732334, 406, 555732335, 409, 555533554, 393645, 555732340, 555533555, 555732341, 446, 451, 456, 460, 555732346, 555732348, 555654194, 555732350, 312715, 503, 555732354, 555732355, 555654206, 549, 555533560, 555732372, 555732373, 604, 555732380, 555732381, 663, 673, 555732388, 555732395, 555533566, 555732396, 555732400, 757, 763, 555533568, 555732407, 555732416, 818, 823, 825, 826, 833, 840, 555732421, 555732422, 555732423, 854, 856, 859, 861, 890, 895, 555732434, 906, 908, 909, 910, 912, 913, 914, 915, 555732436, 394135, 394137, 394138, 394139, 394141, 555732438, 5557

In [8]:
merger.df

transition_0  transition_1  transition_2  transition_3  \
cep_id qid                                                           
1      0    8.957921e+02  0.000000e+00           0.0           0.0   
2      1    6.308205e+10  2.213737e+11           0.0           0.0   
       2    6.308205e+10  3.943634e+11           0.0           0.0   
       3    0.000000e+00  9.987597e+07           0.0           0.0   
       4    6.310517e+10  6.332017e+11           0.0           0.0   
...                  ...           ...           ...           ...   
464706 354  9.263196e+07  0.000000e+00           0.0           0.0   
464707 155  5.789740e+04  0.000000e+00           0.0           0.0   
464708 155  2.222524e+07  0.000000e+00           0.0           0.0   
464709 155  1.856216e+06  0.000000e+00           0.0           0.0   
464710 354  1.262387e+06  0.000000e+00           0.0           0.0   

            transition_4  transition_5  transition_6  transition_7  \
cep_id qid                                                           
1      0             0.0      0.000000           0.0           0.0   
2      1             0.0      0.000000           0.0           0.0   
       2             0.0      0.000000           0.0           0.0   
       3             0.0      0.000000           0.0           0.0   
       4             0.0      0.000000           0.0           0.0   
...                  ...           ...           ...           ...   
464706 354           0.0      0.000000           0.0           0.0   
464707 155           0.0      0.000000           0.0           0.0   
464708 155           0.0  62972.843269           0.0           0.0   
464709 155           0.0      0.000000           0.0           0.0   
464710 354           0.0      0.000000           0.0           0.0   

            transition_8  transition_9  transition_10  country  \
cep_id qid                                                       
1      0             0.0           0.0       0.000000      171   
2      1             0.0           0.0       0.000000        1   
       2             0.0           0.0       0.000000        1   
       3             0.0           0.0       0.000000        1   
       4             0.0           0.0       0.000000        1   
...                  ...           ...            ...      ...   
464706 354           0.0           0.0       0.000000      303   
464707 155           0.0           0.0       0.000000      303   
464708 155           0.0           0.0   18095.571529      303   
464709 155           0.0           0.0       0.000000      303   
464710 354           0.0           0.0       0.000000      303   

                                 country_name  iso3    eco  \
cep_id qid                                                   
1      0                            Lithuania   LTU  80412   
2      1    Area Beyond National Jurisdiction  ABNJ      1   
       2    Area Beyond National Jurisdiction  ABNJ      1   
       3    Area Beyond National Jurisdiction  ABNJ      1   
       4    Area Beyond National Jurisdiction  ABNJ      1   
...                                       ...   ...    ...   
464706 354                           Zimbabwe   ZWE  31006   
464707 155                           Zimbabwe   ZWE  31006   
464708 155                           Zimbabwe   ZWE  31006   
464709 155                           Zimbabwe   ZWE  31006   
464710 354                           Zimbabwe   ZWE  31006   

                                                    eco_name  is_marine  \
cep_id qid                                                                
1      0                      Central European mixed forests      False   
2      1                                           Antarctic       True   
       2                                           Antarctic       True   
       3                                           Antarctic       True   
       4                                           Antarctic       True   

 # Checking area of Slovenia (incorrect shows up as double the area)

In [9]:
tdf = merger.df
#for each row add up all the transition bands (column 0 to 10) and store in a new column
tdf['total'] = tdf.iloc[:, 0:11].sum(axis=1)
# group by cep_id and sum the total areas
tdf = tdf.groupby('cep_id').sum()
tdf.reset_index(inplace=True)
tdf.head()

,cep_id,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,...,country,country_name,iso3,eco,eco_name,is_marine,pa,pa_name,is_protected,total
0,1,8.957921e+02,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,171,Lithuania,LTU,80412,Central European mixed forests,0,0,0,0,8.957921e+02
1,2,1.366843e+12,7.194092e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31,Area Beyond National JurisdictionArea Beyond N...,ABNJABNJABNJABNJABNJABNJABNJABNJABNJABNJABNJAB...,31,AntarcticAntarcticAntarcticAntarcticAntarcticA...,31,0,0,0,8.560935e+12
2,4,0.000000e+00,1.836981e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,1,555547601,South Georgia and South Sandwich Islands Marin...,1,1.836981e+06
3,6,6.328776e+11,1.038784e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54,Area Beyond National JurisdictionArea Beyond N...,ABNJABNJABNJABNJABNJABNJABNJABNJABNJABNJABNJAB...,108,Antarctic Polar FrontAntarctic Polar FrontAnta...,54,0,0,0,1.102071e+13
4,7,0.000000e+00,1.167926e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5,Area Beyond National JurisdictionArea Beyond N...,ABNJABNJABNJABNJABNJ,10,Antarctic Polar FrontAntarctic Polar FrontAnta...,5,1729440,Terres Australes FrançaisesTerres Australes Fr...,5,1.167926e+10


In [10]:
tdf[tdf['cep_id'].between(350156, 353740)]['total'].sum()/ 1000000 # area of Slovenia should be around 204,701 km^2

20470.917301811078